In [1]:
import pandas as pd
import openpyxl

# Load year data
year='2022'
d = pd.read_excel(f'{year}_raw.xlsx', engine='openpyxl')

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


# Map+Pie Yes/No data

In [20]:

def select_cols(d,cols):
    '''Select related columns'''
    df= d.iloc[:,[8]+ cols] #8 is Federal district
    return df


def calculate_percentages(df):
    '''Calculate percentage of yes/no per district & question, 
    and averages across all districts'''

    #Count frequencies of responses per question per district
    counts = {}
    first_col = df.columns[0] ##Федеральный округ
    for col in df.columns[1:]:
        counts[col] = df.groupby(first_col)[col].value_counts().fillna(0)
    #Transform to data frame
    counts=pd.DataFrame(counts)
    #Calculate total frequencies of responses for the category per district
    counts['All'] = counts.iloc[:, 1:].sum(axis=1)

    #Get names of the districts
    districts = df[first_col].unique()
    
    #
    district_percentages = {}
    for district in districts:
        #Calculate yes/(yes+no) percentages per column and per disctrict        
        for col in counts.columns:
            da_count = counts.loc[(district, 'Да'), col] if (district, 'Да') in counts.index else 0
            net_count = counts.loc[(district, 'Нет'), col] if (district, 'Нет') in counts.index else 0
            
            if da_count + net_count != 0: 
                district_percentages[(district, col)] = (da_count / (da_count + net_count)) * 100
            elif (district, 'Да') not in counts.index:
                district_percentages[(district, col)] = 0

            else:
                district_percentages[(district, col)] = 100

#         print(data.loc[(district, 'Да')])

    #To dataframe
    df_district_percentages = pd.DataFrame(list(district_percentages.items()), columns=['District_Column', 'Percentage'])
    df_district_percentages[['District', 'Column']] = pd.DataFrame(df_district_percentages['District_Column'].tolist(), 
                                                                              index=df_district_percentages.index)
    df_district_percentages = df_district_percentages.pivot(index='District', columns='Column', values='Percentage')

    #Averages across all districts
    averages = df_district_percentages.mean()
    df_district_percentages.loc['Все'] = averages
    
    return df_district_percentages


In [38]:
d.columns[68:]
#TODO: rename columns to shorter

Index(['Почувствовали ли вы себя как ЛГБТК+ человек в более уязвимом/небезопасном положении после начала вооруженного конфликта между Россией и Украиной 24 февраля 2022 года?',
       'Заметили ли вы усиление гомофобных и/или трансфобных настроений в российском обществе  после начала вооруженного конфликта между Россией и Украиной 24 февраля 2022 года?',
       'Заметили ли вы усиление гомофобных и/или трансфобных настроений в вашем ближайшем окружении  после начала вооруженного конфликта между Россией и Украиной 24 февраля 2022 года?',
       'Повлиял ли  вооруженный конфликт между Россией и Украиной в 2022 году на доступ к значимым для вас лекарствам или препаратам (например, гормональная терапия, антиретровирусная терапия, антидепрессанты и др.)?',
       'Ухудшил ли вооруженный конфликт между Россией и Украиной в 2022 году ваше экономическое положение?',
       'Коснулась ли вас лично объявленная в России мобилизация?',
       'Как именно вас коснулась мобилизация?',
       'Вынуди

In [39]:
#Set columns for categories
cols_economy = [20,24,25]
cols_discrimination = [28,30,43,51,52,55,57,58,63,65]
cols_violence = list(range(32, 36))+list(range(37, 42))
cols_wareffect = [68,70,72]

#Subset data
df_economy = select_cols(d,cols_economy)
df_discrimination = select_cols(d,cols_discrimination)
df_violence = select_cols(d,cols_violence)
df_wareffect = select_cols(d,cols_wareffect)

#Calcuate percentages
res_economy = calculate_percentages(df_economy)
res_discrimination = calculate_percentages(df_discrimination)
res_violence = calculate_percentages(df_violence)
res_wareffect = calculate_percentages(df_wareffect)

In [41]:
res_economy.to_csv('res_economy.csv')
res_discrimination.to_csv('res_discrimination.csv')
res_violence.to_csv('res_violence.csv')
res_wareffect.to_csv('res_wareffect.csv')

In [26]:

res_discrimination

Column,All,Были ли у вас в 2022 году какие-либо проблемы при поиске места работы/учебы из-за сексуальной ориентации и/или трансгендерности?,Приходилось ли вам в 2022 году сталкиваться с насильственными/ дискриминирующими действиями врачей на почве гомофобии/трансфобии?,Приходилось ли вам в 2022 году сталкиваться с отказом в предоставлении медицинских услуг из-за вашей сексуальной ориентации и/или трансгендерностии?,Приходилось ли вам в 2022 году сталкиваться со случаями отказа в доступе к государственным услугам из-за сексуальной ориентации и/или трансгендерности?,Приходилось ли вам в 2022 году терять работу/место учёбы из-за сексуальной ориентации и/или трансгендерности?,"Сталкивались ли вы в 2022 году с ситуацией, когда вас лишили жилья из-за вашей сексуальной ориентации и/или трансгендерности?","Сталкивались ли вы в 2022 году с трудностями в поиске жилья в связи с вашей сексуальной ориентацией, трансгендерностью или гендерной неконформностью?",Сталкивались ли вы в 2022 году с трудностями при взаимодействии с детскими образовательными или медицинскими учреждениями из-за ваших сексуальной ориентации и/или трансгендерности?,Сталкивались ли вы в 2022 году с трудностями при взаимодействии с органами опеки из-за ваших сексуальной ориентации и/или трансгендерности?
District,,,,,,,,,,
Дальневосточный федеральный округ,6.727480,32.500000,34.375000,9.375000,2.970297,7.567568,5.445545,5.940594,18.181818,NaN
Приволжский федеральный округ,4.303227,18.446602,15.789474,4.678363,0.778643,4.566745,3.448276,6.562848,NaN,1.923077
Северо-Западный федеральный округ,4.674412,19.607843,17.330211,4.683841,1.754386,3.764861,2.819549,6.892231,6.504065,3.252033
Северо-Кавказский федеральный округ,8.163265,32.258065,50.000000,25.000000,NaN,14.285714,3.389831,11.864407,NaN,NaN
Сибирский федеральный округ,3.824092,23.507463,10.891089,5.940594,1.335559,3.846154,4.006678,4.841402,NaN,NaN
Уральский федеральный округ,4.896339,21.266968,19.767442,6.976744,1.353965,4.048583,4.061896,7.350097,8.000000,NaN
Центральный федеральный округ,5.255053,19.520852,19.379845,6.976744,2.185315,4.524469,3.321678,7.167832,10.365854,3.048780
Южный федеральный округ,5.469388,20.000000,18.367347,16.326531,1.433692,8.270677,3.225806,5.017921,NaN,8.333333
Все,5.414157,23.388474,23.237551,9.994727,1.687408,6.359346,3.714907,6.954666,10.762934,4.139306


In [23]:
res_economy

Column,All,"Есть ли у вас вспомогательные источники какой-либо финансовой поддержки (например, родные, партнер/ка, друзья регулярно или хотя бы иногда помогают вам материально)?","Есть ли у вас иждивенцы - те, кто не имеет собственного источника дохода и кого вы финансово поддерживаете на постоянной основе, например, дети, родственники, партнёр(ка)?",Есть ли у вас постоянный источник дохода?,"Приходилось ли вам в 2022 году получать оплату за секс, эскорт, работу моделью в порнографии или вебкам-моделью?"
District,,,,,
Дальневосточный федеральный округ,44.554455,65.841584,15.346535,64.851485,2.970297
Приволжский федеральный округ,48.571429,65.105909,12.235818,72.858732,7.786429
Северо-Западный федеральный округ,47.930602,64.949749,15.476190,74.937343,3.947368
Северо-Кавказский федеральный округ,48.022599,76.271186,20.338983,61.016949,6.779661
Сибирский федеральный округ,46.993318,63.377926,11.185309,69.282137,8.347245
Уральский федеральный округ,47.388781,63.056093,11.411992,71.953578,7.156673
Центральный федеральный округ,47.129117,64.435696,16.608392,72.858392,4.108392
Южный федеральный округ,48.028674,73.118280,12.544803,65.591398,5.376344
Все,47.327372,67.019553,14.393503,69.168752,5.809051


In [24]:
res_violence

Column,All,Подвергались ли вы в 2022 году сексуализированному насилию (в том числе домогательствам и попыткам изнасилования) из-за вашей сексуальной ориентации и/или трансгендерности?,Подвергались ли вы в 2022 году физическому насилию из-за сексуальной ориентации и/или трансгендерности?,Сталкивались ли вы в 2022 году с грабежом по типу “подставных свиданий” ?,"Сталкивались ли вы в 2022 году с домашним насилием из-за сексуальной ориентации и/или трансгендерности (со стороны родителей, родственников, супругов, партнёров)?",Сталкивались ли вы в 2022 году с угрозами физическим насилием из-за сексуальной ориентации и/или трансгендерности?,Сталкивались ли вы в 2022 году с шантажом/вымогательством с угрозой раскрытия информации о вашей сексуальной ориентации или трансгендерности?,Сталкивались ли вы в 2022 году со случаями незаконного использования ваших персональных данных из-за вашей сексуальной ориентации и/или трансгендерности?,"Сталкивались ли вы в 2022 году со случаями причинения вреда вашему имуществу, уничтожения, кражи/хищения имущества из-за вашей сексуальной ориентации и/или трансгендерности?",Сталкивались ли именно вы в 2022 году с преследованиями в интернете из-за вашей сексуальной ориентации и/или трансгендерности?
District,,,,,,,,,,
Дальневосточный федеральный округ,14.047030,6.435644,8.910891,5.940594,17.326733,26.732673,18.316832,13.366337,7.920792,16.336634
Приволжский федеральный округ,12.388765,7.675195,7.675195,4.226919,13.459399,21.134594,15.795328,11.902113,7.675195,17.241379
Северо-Западный федеральный округ,9.249687,7.080201,5.764411,2.380952,14.285714,17.481203,8.959900,7.769424,5.137845,10.902256
Северо-Кавказский федеральный округ,11.864407,6.779661,11.864407,NaN,20.338983,22.033898,8.474576,13.559322,13.559322,10.169492
Сибирский федеральный округ,14.336394,10.350584,10.183639,6.176962,15.525876,27.378965,17.863105,13.522538,6.343907,17.529215
Уральский федеральный округ,13.660542,10.444874,9.090909,4.255319,15.280464,26.112186,18.762089,12.572534,6.576402,15.280464
Центральный федеральный округ,10.484047,8.304196,7.211538,3.452797,14.641608,18.444056,12.150350,9.353147,4.982517,12.543706
Южный федеральный округ,11.962366,9.677419,6.093190,3.584229,15.053763,22.580645,14.695341,9.677419,4.301075,16.129032
Все,12.249155,8.343472,8.349273,4.288253,15.739068,22.737278,14.377190,11.465354,7.062132,14.516522


In [40]:
res_wareffect

Column,All,Заметили ли вы усиление гомофобных и/или трансфобных настроений в вашем ближайшем окружении после начала вооруженного конфликта между Россией и Украиной 24 февраля 2022 года?,Почувствовали ли вы себя как ЛГБТК+ человек в более уязвимом/небезопасном положении после начала вооруженного конфликта между Россией и Украиной 24 февраля 2022 года?,Ухудшил ли вооруженный конфликт между Россией и Украиной в 2022 году ваше экономическое положение?
District,,,,
Дальневосточный федеральный округ,55.470738,45.685279,77.948718,65.306122
Приволжский федеральный округ,55.335968,42.051860,78.354002,68.665158
Северо-Западный федеральный округ,52.681992,35.421995,81.325685,69.897959
Северо-Кавказский федеральный округ,59.322034,49.152542,83.050847,69.491525
Сибирский федеральный округ,53.689567,41.595925,75.809199,65.762712
Уральский федеральный округ,51.144279,35.742972,75.984252,66.272189
Центральный федеральный округ,56.197058,39.703903,80.656610,72.485600
Южный федеральный округ,59.706960,48.905109,82.310469,70.588235
Все,55.443574,42.282448,79.429973,68.558688


In [ ]:
#TODO: NaN checker


# Sample stats

In [ ]:
#TODO: Данные по выборке


# Opennes

In [ ]:
#TODO: Данные по открытости
# Взять один показатель (один вопрос) и по нему посмотреть по регионам.

# Cis/Trans subsets for Yes/No Map

In [48]:
df_economy
d.loc[:,d.columns[2]]

0        Да
1        Да
2       Нет
3        Да
4        Да
       ... 
6434    Нет
6435    Нет
6436     Да
6437     Да
6438    Нет
Name: Идентифицируете ли вы себя как трансгендерного человека? , Length: 6439, dtype: object

In [ ]:
#TODO: **По всем категориям** 
# Возможность посмотреть разницу между цисгендерными и трансгендерными людьми - переключатель


def select_rows_cols(d, column_name, value, cols):
    '''Select related rows & columns'''
    df = d[d[column_name] == value]
    df = select_cols(df,cols)
    return df

#Subset data
column_name = 'Идентифицируете ли вы себя как трансгендерного человека? '
value1 = 'Да'
value2 = 'Нет'
df_economy_cis = select_rows_cols(d,column_name, value2,cols_economy)
df_discrimination_cis = select_rows_cols(d,column_name, value2, cols_discrimination)
df_violence_cis = select_rows_cols(d,column_name, value2, cols_violence)
df_wareffect_cis = select_rows_cols(d,column_name, value2,cols_wareffect)
df_economy_trans = select_rows_cols(d,column_name, value1, cols_economy)
df_discrimination_trans = select_rows_cols(d,column_name, value1, cols_discrimination)
df_violence_trans = select_rows_cols(d,column_name, value1, cols_violence)
df_wareffect_trans = select_rows_cols(d,column_name, value1, cols_wareffect)

#Calcuate percentages
res_economy_cis = calculate_percentages(df_economy_cis)
res_discrimination_cis = calculate_percentages(df_discrimination_cis)
res_violence_cis = calculate_percentages(df_violence_cis)
res_wareffect_cis = calculate_percentages(df_wareffect_cis)

res_economy_trans = calculate_percentages(df_economy_trans)
res_discrimination_trans = calculate_percentages(df_discrimination_trans)
res_violence_trans = calculate_percentages(df_violence_trans)
res_wareffect_trans = calculate_percentages(df_wareffect_trans)

